In [49]:
import pandas as pd
import numpy as np
import math

In [3]:
base_dir = "Data/MLB_2017/"
f = pd.read_csv(base_dir+"MLB_PitchFX_2017_RegularSeason.csv")

In [76]:
def distance(pitch_id, batter_id,balls,strikes, pitch_count, inning, c=10):
    def id_dis(id1, id2):
        return 0 if id1 == id2 else c
    def dis(x):
        sqdis = (x['balls']-balls)**2+\
        (x['strikes']-strikes)**2+\
        (x['pitch_count']-pitch_count)**2+\
        (x['inning']-inning)**2+\
        id_dis(pitch_id,x['pitcher'])+\
        id_dis(batter_id,x['batter'])
        return sqdis
    return dis
def predict(all_data,pitch_id, batter_id,balls,strikes, pitch_count,inning,
            num_of_neighbor=5, c=10):
    f_l=distance(pitch_id,batter_id,balls,strikes,pitch_count,inning,c=c)
    all_data["dis"]=all_data.apply(f_l,axis=1)
    sort_results=all_data.sort_values(by=["dis"])
    select_umpcall = sort_results[0:num_of_neighbor]["umpcall"]
    
    #count weighted states
    X = 0
    S = 0
    B = 0
    i = 0
    for state in select_umpcall:
        tmp = math.exp(-i/num_of_neighbor*3)
        if state == "X":
            X+=tmp
        elif state == "S":
            S+=tmp
        else:
            B+=tmp
        i+=1
    XBS = X+S+B
    print(sort_results[["dis","umpcall"]])
    return {"S": S/XBS,
            "B": B/XBS,
            "X": X/XBS}

In [55]:
f_l=distance("greinza01","spande01",1,1,3,1,c=10)
f["dis"]=f.apply(f_l,axis=1)

In [74]:
select_umpcall = f.sort_values(by=["dis"])[0:10]["umpcall"]
print(f.sort_values(by=["dis"])[0:10][["dis","umpcall"]])

        dis umpcall
512       0       X
511       2       B
510       5       S
509      11       S
181565   11       S
500322   11       X
124121   11       S
311062   11       B
135924   11       S
135923   11       S


In [78]:
print(predict(f,"holdejo02","sanchga02",1,2,16,7,num_of_neighbor=7, c=10))

          dis umpcall
632583     10       B
512        10       X
159171     11       S
356268     11       S
148957     11       S
148958     11       S
159172     11       S
152286     11       S
356269     11       B
124121     11       S
632582     12       B
632584     12       B
511        12       B
462925     12       X
318223     12       S
695136     12       S
174540     12       X
124122     12       B
124120     12       S
503341     13       X
148956     13       B
221048     13       B
695135     13       S
152285     13       S
236379     13       S
159252     14       X
318222     14       S
327036     14       X
291370     14       S
266045     14       S
...       ...     ...
111828  11470       B
122527  11470       S
194812  11471       S
138352  11474       S
278992  11474       B
388081  11475       B
99671   11475       S
436560  11475       S
251079  11477       S
111829  11686       B
99672   11687       B
436561  11687       S
138353  11689       S
388082  11